In [1]:
# Run this cell to mount your Google Drive.

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
pth = '/content/drive/MyDrive/Colab Notebooks/Thesis'

# Gin Rummy

## Imports

In [3]:
#-------------------------------------------------------------------------------
# The following code was originally written by Todd Neller in Java.
# It was translated into Python by Anthony Hein.
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# A class for modeling a game of Gin Rummy
# @author Todd W. Neller
# @version 1.0
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Copyright (C) 2020 Todd Neller
#
# This program is free software; you can redistribute it and/or
# modify it under the terms of the GNU General Public License
# as published by the Free Software Foundation; either version 2
# of the License, or (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# Information about the GNU General Public License is available online at:
#   http://www.gnu.org/licenses/
# To receive a copy of the GNU General Public License, write to the Free
# Software Foundation, Inc., 59 Temple Place - Suite 330, Boston, MA
# 02111-1307, USA.
#-------------------------------------------------------------------------------

%cd /content/drive/My Drive/Colab Notebooks/Thesis/GinRummy
import random
import time
import numpy as np
import os
from Deck import Deck
from GinRummyUtil import GinRummyUtil
from SimpleGinRummyPlayer import SimpleGinRummyPlayer

%cd /content/drive/My Drive/Colab Notebooks/Thesis
#-------------------------------------------------------------------------------

# TRACKING
# Plane (5x52)      Feature
# 0	 currHand       the cards in current player's hand
# 1	 topCard        the top card of the discard pile
# 2	 deadCard       the dead cards: cards in discard pile (excluding the top card)
# 3	 oppCard        opponent known cards: cards picked up from discard pile, but not discarded
# 4	 unknownCard    the unknown cards: cards in stockpile or in opponent hand (but not known)

# Action ID         Action
# 0	                score_player_0_action
# 1	                score_player_1_action
# 2	                draw_card_action
# 3	                pick_up_discard_action
# 4	                declare_dead_hand_action
# 5	                gin_action
# 6 - 57	        discard_action
# 58 - 109	        knock_action

# Knock_bin
# Action ID         Action
# 0	                No Knock
# 1	                Knock

def one_hot(cards):
    ret = np.zeros(52)
    for card in cards:
        ret[card.getId()] = 1
    return ret

def un_one_hot(arr):
    rankNames = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K"]
    suitNames = ['S', 'H', 'D', 'C']
    ret = []
    for i in range(len(arr)):
        if arr[i] != 0:
            ret.append(rankNames[i%13] + suitNames[i//13])
    return ret

#-------------------------------------------------------------------------------

/content/drive/My Drive/Colab Notebooks/Thesis/GinRummy
/content/drive/My Drive/Colab Notebooks/Thesis


## Game Definition

In [4]:
class GinRummyGame:

    # Hand size (before and after turn). After draw and before discard there is one extra card.
    HAND_SIZE = 10;

    # Whether or not to print information during game play
    playVerbose = False;

    # Two Gin Rummy players numbered according to their array index.
    players = [];

    # Set whether or not there is to be printed output during gameplay.
    def setPlayVerbose(playVerbose):
        GinRummyGame.playVerbose = playVerbose
    
    #-------------------------------- appendStates --------------------------------#
    # 2020-12-20: Define a method to append states
    def appendState(self, discards, oppCard):
        currHand = one_hot(GinRummyGame.players[0].cards)
        topCard = np.zeros(52)
        if len(discards) > 0:
            topCard[discards[-1].getId()] = 1
        deadCard = np.zeros(52)
        for d in range(len(discards) - 1):
            deadCard[discards[d].getId()] = 1
        unknownCard = np.ones(52) - currHand - topCard - deadCard - oppCard
        states.append(np.array([currHand, topCard, deadCard, oppCard, unknownCard]).flatten())
    #------------------------------------------------------------------------------#

    # Create a GinRummyGame with two given players
    def __init__(self, player0, player1):
        GinRummyGame.players.extend([player0, player1])

    # Play a game of Gin Rummy and return the winning player number 0 or 1.
    # @return the winning player number 0 or 1

    # 2020-12-20: Add parameter to select what data to generate
    def play(self,state='all',action='all'):
        scores = [0, 0]
        hands = []
        hands.extend([[], []])

        startingPlayer = random.randrange(2);

        # while game not over
        while scores[0] < GinRummyUtil.GOAL_SCORE and scores[1] < GinRummyUtil.GOAL_SCORE:

            currentPlayer = startingPlayer
            opponent = (1 if currentPlayer == 0 else 0)

            # get shuffled deck and deal cards
            deck = Deck.getShuffle(random.randrange(10 ** 8))
            hands[0] = []
            hands[1] = []
            for i in range(2 * GinRummyGame.HAND_SIZE):
                hands[i % 2] += [deck.pop()]
            for i in range(2):
                GinRummyGame.players[i].startGame(i, startingPlayer, hands[i]);
                if GinRummyGame.playVerbose:
                    print("Player %d is dealt %s.\n" % (i, hands[i]))
            if GinRummyGame.playVerbose:
                print("Player %d starts.\n" % (startingPlayer))
            discards = []
            discards.append(deck.pop())
            if GinRummyGame.playVerbose:
                print("The initial face up card is %s.\n" % (discards[len(discards) - 1]))
            firstFaceUpCard = discards[len(discards) - 1]
            turnsTaken = 0
            knockMelds = None

            # 11/25 - Initial state, prior to any cards
            currHand = np.zeros(52)
            topCard = np.zeros(52)
            deadCard = np.zeros(52)
            oppCard = np.zeros(52)
            unknownCard = np.ones(52)

            # while the deck has more than two cards remaining, play round
            while len(deck) > 2:
#-------------------------------------------------------------- BPBD --------------------------------------------------------------#
                drawFaceUp = False
                faceUpCard = discards[len(discards) - 1]

                # offer draw face-up iff not 3rd turn with first face up card (decline automatically in that case)
                if not (turnsTaken == 2 and faceUpCard == firstFaceUpCard):
                    # both players declined and 1st player must draw face down
                    drawFaceUp = GinRummyGame.players[currentPlayer].willDrawFaceUpCard(faceUpCard)
                    if GinRummyGame.playVerbose and not drawFaceUp and faceUpCard == firstFaceUpCard and turnsTaken < 2:
                        print("Player %d declines %s.\n" % (currentPlayer, firstFaceUpCard))

                if not (not drawFaceUp and turnsTaken < 2 and faceUpCard == firstFaceUpCard):

                    #------------------------------------ DRAW ------------------------------------#
                    # 2020-12-01  -  Track states BEFORE the player PICKUP BEFORE player DISCARDS (track_bpbd)
                    # Action      -  PickUp from Discard(FaceUp) or Deck (Unknown)
                    # State       -  BPBD -> APBD

                    if (state == 'bpbd' or state == 'all') and currentPlayer == 0:
                        # record current state
                        self.appendState(discards,oppCard)
                        
                        # record current action
                        currAction = np.zeros(110)
                        if drawFaceUp:
                            currAction[3] = 1
                        else:
                            currAction[2] = 1
                        actions.append(currAction)

                    #------------------------------------------------------------------------------#

                    # continue with turn if not initial declined option
                    drawCard = discards.pop() if drawFaceUp else deck.pop()
                    for i in range(2):
                        to_report = drawCard if i == currentPlayer or drawFaceUp else None
                        GinRummyGame.players[i].reportDraw(currentPlayer, to_report)

                    if GinRummyGame.playVerbose:
                        print("Player %d draws %s.\n" % (currentPlayer, drawCard))
                    hands[currentPlayer].append(drawCard)
#-------------------------------------------------------------- APBD --------------------------------------------------------------#
                    discardCard = GinRummyGame.players[currentPlayer].getDiscard()

                    #-------------------------------- DISCARD/KNOCK -------------------------------#
                    # 2020-11-25  -  Track states AFTER the player PICKUP BEFORE player DISCARDS (track_apbd)
                    # 2020-12-01  -  incorporated the remaining actions to output space
                    # 2020-12-20  -  NOTE: Knock/Gin actions are also APBD states
                    # Action      -  Discard card from player hand
                    # State       -  APBD -> APAD

                    if (state == 'apbd' or state == 'all') and currentPlayer == 0:
                        # record current state
                        self.appendState(discards,oppCard)

                        # record current action as DISCARD*
                        removedCard = np.zeros(52)
                        removedCard[discardCard.getId()] = 1
                        currAction = np.append(np.append(np.zeros(6), removedCard), np.zeros(52))
                        actions.append(currAction)

                    #------------------------------------------------------------------------------#
                    
                    # Track opponent pickup and discard after each discard 
                    if currentPlayer == 1:
                        # Set discarded card to 0 (in case discarded card was seen)
                        oppCard[discardCard.getId()] = 0
                        if drawFaceUp: # if opponent draws TopCard from discard
                            oppCard[drawCard.getId()] = 1


                    if not discardCard in hands[currentPlayer] or discardCard == faceUpCard:
                        print("Player %d discards %s illegally and forfeits.\n" % (currentPlayer, discardCard))
                        return opponent;
                    hands[currentPlayer].remove(discardCard)
                    for i in range(2):
                        GinRummyGame.players[i].reportDiscard(currentPlayer, discardCard)                    
                    if GinRummyGame.playVerbose:
                        print("Player %d discards %s.\n" % (currentPlayer, discardCard))
                    discards.append(discardCard)

#-------------------------------------------------------------- APAD --------------------------------------------------------------#
                    #------------------------------- KNOCK - Binary -------------------------------#
                    # 2020-12-20  -  Track states AFTER the player PICKUP AFTER player DISCARDS
                    # Action      -  Knock (1) or No Knock (0)
                    # State       -  APAD -> Knock/No Knock 
                    if state == 'apad' and currentPlayer == 0:
                        # record current state
                        self.appendState(discards,oppCard)
                        
                        # record current action as not knocked
                        actions.append([1,0])
                        
                    #-------------------------------------------------------------------------------#

                    if GinRummyGame.playVerbose:
                        unmeldedCards = hands[currentPlayer].copy()
                        bestMelds = GinRummyUtil.cardsToBestMeldSets(unmeldedCards)
                        if len(bestMelds) == 0:
                            print("Player %d has %s with %d deadwood.\n" % (currentPlayer, unmeldedCards, GinRummyUtil.getDeadwoodPoints3(unmeldedCards)))
                        else:
                            melds = bestMelds[0]
                            for meld in melds:
                                for card in meld:
                                    unmeldedCards.remove(card)
                            melds.extend(unmeldedCards)
                            print("Player %d has %s with %d deadwood.\n" % (currentPlayer, melds, GinRummyUtil.getDeadwoodPoints3(unmeldedCards)))

#-------------------------------------------------------------- KNOCK --------------------------------------------------------------#
                    # CHECK FOR KNOCK
                    knockMelds = GinRummyGame.players[currentPlayer].getFinalMelds()
                    if knockMelds != None:
                        # player knocked; end of round
                        #--------------------------------- APBD: KNOCK  --------------------------------#
                        # 2020-12-20  -  Differentiate between Discard and Knock
                        #                1. remove action (should be Knock/Gin instead of Discard)
                        #                2. if action is 'discard'
                        #                  a. TRUE: remove state
                        #                  b. FALSE: change action to Knock
                        if (state == 'apbd' or state == 'all') and currentPlayer == 0:
                            # remove current action (since it was DISCARD from APBD DISCARD/KNOCK)
                            removedAction = actions.pop(-1)
                            # remove state if action is Discard
                            if action == 'discard':
                                states.pop(-1)
                            else:
                                # Get removed card from action list
                                removedCard = np.where(removedAction==1)[0][0] - 6
                                knockedCard = np.zeros(52)
                                knockedCard[removedCard] = 1
                                # Record as a Knock action
                                currAction = np.append(np.append(np.zeros(6), np.zeros(52)), knockedCard)
                                actions.append(currAction)
                        
                        #------------------------------------------------------------------------------#

                        #----------------------------- APAD: KNOCK(Binary) -----------------------------#
                        # 2020-12-20  -  replace current action from 'No Knock' to 'Knock'
                        if state == 'apad' and currentPlayer == 0:
                            actions[-1] = [0,1]

                        #-------------------------------------------------------------------------------#
                        break
                    else:
                        # player did not knock
                        #---------------------------------- APBD: KNOCK  --------------------------------#
                        # 2020-12-20  -  remove current state since action was not Knock
                        if (action == 'knock') and currentPlayer == 0:
                            states.pop(-1)
                            actions.pop(-1)
                        
                        #------------------------------------------------------------------------------#
                turnsTaken += 1
                currentPlayer = 1 if currentPlayer == 0 else 0
                opponent = 1 if currentPlayer == 0 else 0

            if knockMelds != None:
                # round didn't end due to non-knocking and 2 cards remaining in draw pile
                # check legality of knocking meld
                handBitstring = GinRummyUtil.cardsToBitstring(hands[currentPlayer])
                unmelded = handBitstring
                for meld in knockMelds:
                    meldBitstring = GinRummyUtil.cardsToBitstring(meld)
                    if (not meldBitstring in GinRummyUtil.getAllMeldBitstrings()) or ((meldBitstring & unmelded) != meldBitstring):
                        # non-meld or meld not in hand
                        print("Player %d melds %s illegally and forfeits.\n" % (currentPlayer, knockMelds))
                        return opponent
                    unmelded &= ~meldBitstring # remove successfully melded cards from

                # compute knocking deadwood
                knockingDeadwood = GinRummyUtil.getDeadwoodPoints1(knockMelds, hands[currentPlayer])
                if knockingDeadwood > GinRummyUtil.MAX_DEADWOOD:
                    print("Player %d melds %s with greater than %d deadwood and forfeits.\n" % (currentPlayer, knockMelds, knockingDeadwood))
                    return opponent

                meldsCopy = []
                for meld in knockMelds:
                    meldsCopy.append(meld.copy())
                for i in range(2):
                    GinRummyGame.players[i].reportFinalMelds(currentPlayer, meldsCopy)
                if GinRummyGame.playVerbose:
                    if knockingDeadwood > 0:
                        print("Player %d melds %s with %d deadwood from %s.\n" % (currentPlayer, knockMelds, knockingDeadwood, GinRummyUtil.bitstringToCards(unmelded)))
                    else:
                        print("Player %d goes gin with melds %s.\n" % (currentPlayer, knockMelds))

                # get opponent meld
                opponentMelds = GinRummyGame.players[opponent].getFinalMelds();
                meldsCopy = []
                for meld in opponentMelds:
                    meldsCopy.append(meld.copy())
                for i in range(2):
                    GinRummyGame.players[i].reportFinalMelds(opponent, meldsCopy)

                # check legality of opponent meld
                opponentHandBitstring = GinRummyUtil.cardsToBitstring(hands[opponent])
                opponentUnmelded = opponentHandBitstring
                for meld in opponentMelds:
                    meldBitstring = GinRummyUtil.cardsToBitstring(meld)
                    if (meldBitstring not in GinRummyUtil.getAllMeldBitstrings()) or ((meldBitstring & opponentUnmelded) != meldBitstring):
                        # non-meld or meld not in hand
                        print("Player %d melds %s illegally and forfeits.\n" % (opponent, opponentMelds))
                        return currentPlayer
                    opponentUnmelded &= ~meldBitstring # remove successfully melded cards from

                if GinRummyGame.playVerbose:
                    print("Player %d melds %s.\n" % (opponent, opponentMelds))

                # lay off on knocking meld (if not gin)
                unmeldedCards = GinRummyUtil.bitstringToCards(opponentUnmelded)
                if knockingDeadwood > 0:
                    # knocking player didn't go gin
                    cardWasLaidOff = False
                    while True:
                        # attempt to lay each card off
                        cardWasLaidOff = False
                        layOffCard = None
                        layOffMeld = None
                        for card in unmeldedCards:
                            for meld in knockMelds:
                                newMeld = meld.copy()
                                newMeld.append(card)
                                newMeldBitstring = GinRummyUtil.cardsToBitstring(newMeld)
                                if newMeldBitstring in GinRummyUtil.getAllMeldBitstrings():
                                    layOffCard = card
                                    layOffMeld = meld
                                    break
                            if layOffCard != None:
                                if GinRummyGame.playVerbose:
                                    print("Player %d lays off %s on %s.\n" % (opponent, layOffCard, layOffMeld))
                                for i in range(2):
                                    GinRummyGame.players[i].reportLayoff(opponent, layOffCard, layOffMeld.copy())
                                unmeldedCards.remove(layOffCard)
                                layOffMeld.append(layOffCard)
                                cardWasLaidOff = True
                                break
                        if not cardWasLaidOff:
                            break

                opponentDeadwood = 0
                for card in unmeldedCards:
                    opponentDeadwood += GinRummyUtil.getDeadwoodPoints2(card)
                if GinRummyGame.playVerbose:
                    print("Player %d has %d deadwood with %s\n" % (opponent, opponentDeadwood, unmeldedCards))
                # compare deadwood and compute new scores
                if knockingDeadwood == 0:
                    # gin round win

                    #---------------------------------- APBD: GIN ----------------------------------#
                    # 2020-12-20  -  Differentiate between Knock and Gin
                    #                1. remove action (should be Gin instead of Knock)
                    #                2. if action is 'discard'
                    #                  a. TRUE: remove state
                    #                  b. FALSE: change action to Gin 
                    if (action == 'knock' or state == 'all') and currentPlayer == 0:
                        # remove current action
                        actions.pop(-1)
                        # remove state if action is Knock
                        if action == 'knock':
                            states.pop(-1)
                        elif state == 'all':
                            # record GIN action if all states
                            currAction = np.zeros(110)
                            currAction[5] = 1
                            actions.append(currAction)
                    
                    #------------------------------------------------------------------------------#

                    #---------------------------- APAD: KNOCK (Binary) -----------------------------#
                    # 2020-12-20  -  remove current action since action should be Gin instead of Knock
                    if state == 'apad' and currentPlayer == 0:
                        states.pop(-1)
                        actions.pop(-1)

                    #-------------------------------------------------------------------------------#
                    # winloss.append(1)
                    scores[currentPlayer] += GinRummyUtil.GIN_BONUS + opponentDeadwood
                    if GinRummyGame.playVerbose:
                        print("Player %d scores the gin bonus of %d plus opponent deadwood %d for %d total points.\n" % \
                        (currentPlayer, GinRummyUtil.GIN_BONUS, opponentDeadwood, GinRummyUtil.GIN_BONUS + opponentDeadwood))

                elif knockingDeadwood < opponentDeadwood:
                    # non-gin round win:
                    # winloss.append(1)
                    scores[currentPlayer] += opponentDeadwood - knockingDeadwood;
                    if GinRummyGame.playVerbose:
                        print("Player %d scores the deadwood difference of %d.\n" % (currentPlayer, opponentDeadwood - knockingDeadwood))

                else:
                    # undercut win for opponent
                    # winloss.append(0)
                    scores[opponent] += GinRummyUtil.UNDERCUT_BONUS + knockingDeadwood - opponentDeadwood;
                    if GinRummyGame.playVerbose:
                        print("Player %d undercuts and scores the undercut bonus of %d plus deadwood difference of %d for %d total points.\n" % \
                        (opponent, GinRummyUtil.UNDERCUT_BONUS, knockingDeadwood - opponentDeadwood, GinRummyUtil.UNDERCUT_BONUS + knockingDeadwood - opponentDeadwood))

                startingPlayer = 1 if startingPlayer == 0 else 0 # starting player alternates

            # If the round ends due to a two card draw pile with no knocking, the round is cancelled.
            else:
                if GinRummyGame.playVerbose:
                    print("The draw pile was reduced to two cards without knocking, so the hand is cancelled.")

            # report final hands
            for i in range(2):
                for j in range(2):
                    GinRummyGame.players[i].reportFinalHand(j, hands[j].copy())

            # score reporting
            if GinRummyGame.playVerbose:
                print("Player\tScore\n0\t%d\n1\t%d\n" % (scores[0], scores[1]))
            for i in range(2):
                GinRummyGame.players[i].reportScores(scores.copy())

        if GinRummyGame.playVerbose:
            print("Player %s wins.\n" % (0 if scores[0] > scores[1] else 1))
        return 0 if scores[0] >= GinRummyUtil.GOAL_SCORE else 1



# Generate Data

## Shared

In [5]:
# Available states-action pairs:
state_action_pair = {'all': 'all', # all actions EXCEPT 0,1,4
                    'bpbd': 'draw', # actions 2/3 
                    'apbd': ['discard', 'knock'], # actions 6-57, 58-109
                    'apad': 'knock_bin'} # binary action

def generateData(pth,state,action,numGames):
    # Multiple non-verbose games
    GinRummyGame.setPlayVerbose(False)
    numP1Wins = 0
    game = GinRummyGame(SimpleGinRummyPlayer(), SimpleGinRummyPlayer())
    if action in state_action_pair[state]:
        for i in range(numGames):
            if i % 100 == 0:
                print("Game ... ", i)
            numP1Wins += game.play(state=state,action=action)
        # Save states and actions
        # Create path if doesn't exist
        state_pth = '{}/data/{}'.format(pth,state)
        if not os.path.exists(state_pth):
            os.mkdir(state_pth)
        action_pth = '{}/{}'.format(state_pth,action)
        if not os.path.exists(action_pth):
            os.mkdir(action_pth)
        data_pth = action_pth
                
        save_str = numGames//1000
        np.save('{}/s_{}k.npy'.format(data_pth,save_str), states)
        np.save('{}/a_{}k.npy'.format(data_pth,save_str), actions)
    else:
        print('illegeal state-action pair')

## After Pickup, After Discard (APAD)

### Knock (Binary)

In [ ]:
state = 'apad'
action = 'knock_bin'
numGames = 2000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 6000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 8000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [2,6,8]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(128146, 260)
(128146, 2)
(381333, 260)
(381333, 2)
(507434, 260)
(507434, 2)


## After Pickup, Before Discard (APBD)

### Discard

In [ ]:
state = 'apbd'
action = 'discard'
numGames = 2000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 6000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 8000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [2,6,8]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(117438, 260)
(117438, 110)
(350710, 260)
(350710, 110)
(469281, 260)
(469281, 110)


### Knock

In [ ]:
state = 'apbd'
action = 'knock'
numGames = 2000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 6000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 8000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [2,6,8]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(9921, 260)
(9921, 110)
(29891, 260)
(29891, 110)
(39783, 260)
(39783, 110)


#### 32K Games

In [ ]:
state = 'apbd'
action = 'knock'
numGames = 32000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
state = 'apbd'
action = 'knock'
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [32]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(158998, 260)
(158998, 110)


#### 40K Games

In [ ]:
state = 'apbd'
action = 'knock'
numGames = 40000
states, actions = [], []
generateData(pth,state,action,numGames)

Game ...  0
Game ...  100
Game ...  200
Game ...  300
Game ...  400
Game ...  500
Game ...  600
Game ...  700
Game ...  800
Game ...  900
Game ...  1000
Game ...  1100
Game ...  1200
Game ...  1300
Game ...  1400
Game ...  1500
Game ...  1600
Game ...  1700
Game ...  1800
Game ...  1900
Game ...  2000
Game ...  2100
Game ...  2200
Game ...  2300
Game ...  2400
Game ...  2500
Game ...  2600
Game ...  2700
Game ...  2800
Game ...  2900
Game ...  3000
Game ...  3100
Game ...  3200
Game ...  3300
Game ...  3400
Game ...  3500
Game ...  3600
Game ...  3700
Game ...  3800
Game ...  3900
Game ...  4000
Game ...  4100
Game ...  4200
Game ...  4300
Game ...  4400
Game ...  4500
Game ...  4600
Game ...  4700
Game ...  4800
Game ...  4900
Game ...  5000
Game ...  5100
Game ...  5200
Game ...  5300
Game ...  5400
Game ...  5500
Game ...  5600
Game ...  5700
Game ...  5800
Game ...  5900
Game ...  6000
Game ...  6100
Game ...  6200
Game ...  6300
Game ...  6400
Game ...  6500
Game ...  6600
Game ..

In [ ]:
state = 'apbd'
action = 'knock'
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [40]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(198987, 260)
(198987, 110)


#### 80K Games

In [6]:
state = 'apbd'
action = 'knock'
numGames = 80000
states, actions = [], []
generateData(pth,state,action,numGames)

Game ...  0
Game ...  100
Game ...  200
Game ...  300
Game ...  400
Game ...  500
Game ...  600
Game ...  700
Game ...  800
Game ...  900
Game ...  1000
Game ...  1100
Game ...  1200
Game ...  1300
Game ...  1400
Game ...  1500
Game ...  1600
Game ...  1700
Game ...  1800
Game ...  1900
Game ...  2000
Game ...  2100
Game ...  2200
Game ...  2300
Game ...  2400
Game ...  2500
Game ...  2600
Game ...  2700
Game ...  2800
Game ...  2900
Game ...  3000
Game ...  3100
Game ...  3200
Game ...  3300
Game ...  3400
Game ...  3500
Game ...  3600
Game ...  3700
Game ...  3800
Game ...  3900
Game ...  4000
Game ...  4100
Game ...  4200
Game ...  4300
Game ...  4400
Game ...  4500
Game ...  4600
Game ...  4700
Game ...  4800
Game ...  4900
Game ...  5000
Game ...  5100
Game ...  5200
Game ...  5300
Game ...  5400
Game ...  5500
Game ...  5600
Game ...  5700
Game ...  5800
Game ...  5900
Game ...  6000
Game ...  6100
Game ...  6200
Game ...  6300
Game ...  6400
Game ...  6500
Game ...  6600
Game ..

In [7]:
state = 'apbd'
action = 'knock'
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [80]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(397966, 260)
(397966, 110)


## Before Pickup, Before Discard (BPBD)

### Draw

In [ ]:
state = 'bpbd'
action = 'draw'
numGames = 2000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 6000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 8000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [2,6,8]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(127338, 260)
(127338, 110)
(381602, 260)
(381602, 110)
(508577, 260)
(508577, 110)


## All

In [ ]:
state = 'all'
action = 'all'
numGames = 2000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 6000
states, actions = [], []
generateData(pth,state,action,numGames)
numGames = 8000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [2,6,8]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(255486, 260)
(255486, 110)
(762526, 260)
(762526, 110)
(1017080, 260)
(1017080, 110)


### 16K All

In [ ]:
state = 'all'
action = 'all'
numGames = 16000
states, actions = [], []
generateData(pth,state,action,numGames)

In [ ]:
state = 'all'
action = 'all'
data_pth = '{}/data/{}/{}'.format(pth,state,action)
for i in [16]:
    s = np.load('{}/s_{}k.npy'.format(data_pth,i))
    a = np.load('{}/a_{}k.npy'.format(data_pth,i))
    print(s.shape)
    print(a.shape)

(2035272, 260)
(2035272, 110)
